In [ ]:
import businessdate as bd
today = bd.BusinessDate()

In [ ]:
from yieldcurves.parametric import NelsonSiegelSvensson
from yieldcurves.tools import ascii_plot, lin

In [ ]:
NelsonSiegelSvensson.download()
f = NelsonSiegelSvensson(date=today-'1d')
ascii_plot(lin(0, 30, 0.025), f)